In [1]:
from util import validate
import pandas as pd

ModuleNotFoundError: No module named 'util'

In [ ]:
resf=pd.read_csv('../data/res_exp_model2.csv',index_col=0).T
resf.columns=['lower_threshold','veritas','score']
validate(resf,C0=1,C1=0.76,C2=1.35,DX=False,score=True,
         outfile='exptvalid.png',
         validation_type='fnrexpt')

In [ ]:
resf=pd.read_csv('../data/res_R20_model2.csv',index_col=0).T
resf.columns=['lower_threshold','veritas','score']
resf['dx'] = [int(x[-1]) for x in resf.index.values]
validate(resf,C0=1,C1=0.76,C2=1.35,DX=True,score=True,
         outfile='ptsdvalid.png',
         validation_type='withdx')

In [ ]:
from truthnet import load_veritas_model
from util import drop_empty_string_keys
import numpy as np
M=load_veritas_model('../veritas_models/veritas_005_SUD.pkl.gz')
df=pd.read_csv('../data/SUD.csv',index_col=0,
            keep_default_na=False,dtype=str)
featurenames=M['model'].feature_names
training_index=M['model'].training_index
test_index=np.array([x for x in df.index.values if x not in training_index])
df_test=df.loc[test_index,:][featurenames]
D=df_test.T.to_dict()
D1={k:drop_empty_string_keys(v) for k,v in D.items()}
D=[{'SUB'+str(k):v} for k,v in D1.items()]

df_train=df.loc[training_index,:][featurenames]
Xt=df_train.values.astype(str)
from truthfinder import reveal
Rsud=reveal(D,M,perturb=0,model_path=False,score=False)
resf=pd.DataFrame({x['subject_id']:[x['lower_threshold'],x['lower_prob'],x['veritas'],x['veritas_prob']] for x in  Rsud[0]}).T
resf.columns=['lower_threshold','lower_p','veritas','veritas_p']
resf=resf.assign(score=resf.lower_threshold)

In [ ]:
validate(resf,C0=.3,C1=0.45,C2=1,DX=False,score=False,
         outfile='sudvalid.png',
         validation_type='noscore')

In [ ]:
! pwd

In [ ]:
! ls ../truthnet.py